# 1. Importando Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier


# 2. Importando o conjunto de dados

In [3]:
df = pd.read_csv("SPDadosCriminais_2023.csv")

C:\Users\T420_i5-2520M\AppData\Local\Temp\ipykernel_1620\1001618688.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("SPDadosCriminais_2023.csv")


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618397 entries, 0 to 618396
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   NOME_DEPARTAMENTO                618395 non-null  object 
 1   NOME_SECCIONAL                   618395 non-null  object 
 2   NOME_DELEGACIA                   618395 non-null  object 
 3   NOME_MUNICIPIO                   618395 non-null  object 
 4   NUM_BO                           618397 non-null  object 
 5   ANO_BO                           618397 non-null  int64  
 6   DATA_REGISTRO                    618396 non-null  object 
 7   DATA_OCORRENCIA_BO               616258 non-null  object 
 8   HORA_OCORRENCIA_BO               432730 non-null  object 
 9   DESC_PERIODO                     185667 non-null  object 
 10  DESCR_SUBTIPOLOCAL               618397 non-null  object 
 11  BAIRRO                           608728 non-null  object 
 12  LO

# 3. Processamento de Dados

## 3.1 Excluindo atributos não relevantes ao modelo

In [5]:
df.drop(['NOME_DEPARTAMENTO', 'NOME_SECCIONAL', 'NOME_DELEGACIA', 'NUM_BO', 'ANO_BO', 'DESC_PERIODO', 'DESCR_SUBTIPOLOCAL', 'NOME_DELEGACIA_CIRCUNSCRIÇÃO', 'NOME_DEPARTAMENTO_CIRCUNSCRIÇÃO',
 'NOME_SECCIONAL_CIRCUNSCRIÇÃO', 'DESCR_CONDUTA', 'MES_ESTATISTICA', 'ANO_ESTATISTICA'], axis=1, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618397 entries, 0 to 618396
Data columns (total 12 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   NOME_MUNICIPIO                618395 non-null  object 
 1   DATA_REGISTRO                 618396 non-null  object 
 2   DATA_OCORRENCIA_BO            616258 non-null  object 
 3   HORA_OCORRENCIA_BO            432730 non-null  object 
 4   BAIRRO                        608728 non-null  object 
 5   LOGRADOURO                    618397 non-null  object 
 6   NUMERO_LOGRADOURO             516929 non-null  object 
 7   LATITUDE                      556799 non-null  float64
 8   LONGITUDE                     556799 non-null  float64
 9   NOME_MUNICIPIO_CIRCUNSCRIÇÃO  618397 non-null  object 
 10  RUBRICA                       618383 non-null  object 
 11  NATUREZA_APURADA              618397 non-null  object 
dtypes: float64(2), object(10)
memory usage: 56.6

## 3.2 Filtrando dados nulos ou inúteis

In [7]:
df = df[df["LOGRADOURO"] != "VEDAÇÃO DA DIVULGAÇÃO DOS DADOS RELATIVOS"]

In [8]:
from pprint import pprint
pprint(df["RUBRICA"].unique())

array(['Furto (art. 155)',
       'Homicídio culposo na direção de veículo automotor (Art. 302)',
       'Homicídio (art. 121)', 'Lesão corporal (art. 129)',
       'Lesão corporal culposa na direção de veículo automotor (Art. 303)',
       'Porte ilegal de arma de fogo de uso permitido (Art. 14)',
       'Posse ou porte ilegal de arma de fogo de uso restrito (Art. 16)',
       'Drogas para consumo pessoal sem autorização ou em desacordo (Art.28,caput)',
       'Roubo (art. 157)',
       'Drogas sem autorização ou em desacordo (Art.33, caput)',
       'Localização/Apreensão de objeto', 'Outros não criminal',
       'Associarem-se duas ou mais pessoas - arts. 33, caput e § 1o, e 34 (Art.35,caput)',
       'Comunicação de óbito', 'Entrega de objeto localizado/apreendido',
       'Porte de arma (art. 19)', 'Atropelamento', 'Morte suspeita',
       'Capotamento', 'Desobediência (art. 330)',
       'Adulteração de sinal identificador de veículo automotor (art. 311)',
       'Furto de coisa 

## 3.3 Filtrando crimes não devidamente enquadrados no Código Penal

In [9]:
excluir = ['Localização/Apreensão de objeto', 'Outros não criminal', 'Comunicação de óbito', 'Entrega de objeto localizado/apreendido', 'Atropelamento', 'Morte suspeita', 'Capotamento','Colisão', 'Choque','Engavetamento','Localização/Apreensão e Entrega de objeto','Entrega de veículo localizado/apreendido', 'Cumprimento de mandado de prisão temporária',
       'Localização/Apreensão de veículo','Auto lesão', 'Localização/Apreensão e Entrega de veículo','Cumprimento de mandado de busca e apreensão', 'Entrada ilegal de aparelho movél de comunicação em estabelecimento prisional', 'Abalroamento', 'Suicídio consumado','homicídio culposo','Apreensão de Adolescente']

df = df[~df["RUBRICA"].isin(excluir)]

In [10]:
df["RUBRICA"].unique()

array(['Furto (art. 155)',
       'Homicídio culposo na direção de veículo automotor (Art. 302)',
       'Homicídio (art. 121)', 'Lesão corporal (art. 129)',
       'Lesão corporal culposa na direção de veículo automotor (Art. 303)',
       'Porte ilegal de arma de fogo de uso permitido (Art. 14)',
       'Posse ou porte ilegal de arma de fogo de uso restrito (Art. 16)',
       'Drogas para consumo pessoal sem autorização ou em desacordo (Art.28,caput)',
       'Roubo (art. 157)',
       'Drogas sem autorização ou em desacordo (Art.33, caput)',
       'Associarem-se duas ou mais pessoas - arts. 33, caput e § 1o, e 34 (Art.35,caput)',
       'Porte de arma (art. 19)', 'Desobediência (art. 330)',
       'Adulteração de sinal identificador de veículo automotor (art. 311)',
       'Furto de coisa comum (art. 156)',
       'Extorsão mediante seqüestro (art. 159)',
       'Favorecimento pessoal (art. 348)', 'Ameaça (art. 147)',
       'Fuga de local de acidente (Art. 305)', nan,
       'Embr

## 3.4 Excluindo valores nulos

In [11]:
df.dropna(inplace=True)

In [12]:
df. head(3)

,NOME_MUNICIPIO,DATA_REGISTRO,DATA_OCORRENCIA_BO,HORA_OCORRENCIA_BO,BAIRRO,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,NOME_MUNICIPIO_CIRCUNSCRIÇÃO,RUBRICA,NATUREZA_APURADA
5,S.PAULO,01/01/2023,12/31/2022,09:00:00 PM,SE,AVENIDA DO ESTADO,4021.0,-23.552728,-46.626623,S.PAULO,Furto (art. 155),FURTO - OUTROS
8,S.PAULO,01/01/2023,01/01/2023,07:45:00 AM,BRÁS,AVENIDA DO ESTADO,4127.0,-23.551932,-46.625661,S.PAULO,Furto (art. 155),FURTO - OUTROS
10,S.PAULO,12/31/2022,12/31/2022,04:20:00 PM,SE,RUA DO GLICERIO,90.0,-23.553375,-46.627812,S.PAULO,Furto (art. 155),FURTO - OUTROS


## 3.5 Obtendo valores atomizados a partir de datas

In [13]:
df["DATA_REGISTRO"] = pd.to_datetime(df["DATA_REGISTRO"])
df["DATA_OCORRENCIA_BO"] = pd.to_datetime(df["DATA_OCORRENCIA_BO"])

In [14]:
df["DIAS_REGISTRO"] = (df["DATA_REGISTRO"] - df["DATA_OCORRENCIA_BO"]).astype(str)

In [15]:
df["DIAS_REGISTRO"] = df["DIAS_REGISTRO"].str[0]

In [16]:
df["MES_REGISTRO"] = df["DATA_REGISTRO"].dt.month
df["DIA_REGISTRO"] = df["DATA_REGISTRO"].dt.day

In [17]:
df["MES_OCORRENCIA"] = df["DATA_OCORRENCIA_BO"].dt.month
df["DIA_OCORRENCIA"] = df["DATA_OCORRENCIA_BO"].dt.day

In [18]:
df["HORA_OCORRENCIA_BO"] = df["HORA_OCORRENCIA_BO"].str[:2]

In [19]:
df.head(3)

,NOME_MUNICIPIO,DATA_REGISTRO,DATA_OCORRENCIA_BO,HORA_OCORRENCIA_BO,BAIRRO,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,NOME_MUNICIPIO_CIRCUNSCRIÇÃO,RUBRICA,NATUREZA_APURADA,DIAS_REGISTRO,MES_REGISTRO,DIA_REGISTRO,MES_OCORRENCIA,DIA_OCORRENCIA
5,S.PAULO,2023-01-01,2022-12-31,09,SE,AVENIDA DO ESTADO,4021.0,-23.552728,-46.626623,S.PAULO,Furto (art. 155),FURTO - OUTROS,1,1,1,12,31
8,S.PAULO,2023-01-01,2023-01-01,07,BRÁS,AVENIDA DO ESTADO,4127.0,-23.551932,-46.625661,S.PAULO,Furto (art. 155),FURTO - OUTROS,0,1,1,1,1
10,S.PAULO,2022-12-31,2022-12-31,04,SE,RUA DO GLICERIO,90.0,-23.553375,-46.627812,S.PAULO,Furto (art. 155),FURTO - OUTROS,0,12,31,12,31


## 3.6 Reorganizando o conjunto de dados

In [20]:
df = df[['NOME_MUNICIPIO', 'DIAS_REGISTRO', 'MES_REGISTRO', 'DIA_REGISTRO', 'MES_OCORRENCIA', 'DIA_OCORRENCIA',
        "HORA_OCORRENCIA_BO", 'BAIRRO', 'LOGRADOURO',
       'NUMERO_LOGRADOURO', 'LATITUDE', 'LONGITUDE',
       'NOME_MUNICIPIO_CIRCUNSCRIÇÃO', 'NATUREZA_APURADA', 'RUBRICA']]

In [21]:
df.head(3)

,NOME_MUNICIPIO,DIAS_REGISTRO,MES_REGISTRO,DIA_REGISTRO,MES_OCORRENCIA,DIA_OCORRENCIA,HORA_OCORRENCIA_BO,BAIRRO,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,NOME_MUNICIPIO_CIRCUNSCRIÇÃO,NATUREZA_APURADA,RUBRICA
5,S.PAULO,1,1,1,12,31,09,SE,AVENIDA DO ESTADO,4021.0,-23.552728,-46.626623,S.PAULO,FURTO - OUTROS,Furto (art. 155)
8,S.PAULO,0,1,1,1,1,07,BRÁS,AVENIDA DO ESTADO,4127.0,-23.551932,-46.625661,S.PAULO,FURTO - OUTROS,Furto (art. 155)
10,S.PAULO,0,12,31,12,31,04,SE,RUA DO GLICERIO,90.0,-23.553375,-46.627812,S.PAULO,FURTO - OUTROS,Furto (art. 155)


## 3.7 Transformando valores em formato de cadeia em valores numéricos

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327218 entries, 5 to 618396
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   NOME_MUNICIPIO                327218 non-null  object 
 1   DIAS_REGISTRO                 327218 non-null  object 
 2   MES_REGISTRO                  327218 non-null  int64  
 3   DIA_REGISTRO                  327218 non-null  int64  
 4   MES_OCORRENCIA                327218 non-null  int64  
 5   DIA_OCORRENCIA                327218 non-null  int64  
 6   HORA_OCORRENCIA_BO            327218 non-null  object 
 7   BAIRRO                        327218 non-null  object 
 8   LOGRADOURO                    327218 non-null  object 
 9   NUMERO_LOGRADOURO             327218 non-null  object 
 10  LATITUDE                      327218 non-null  float64
 11  LONGITUDE                     327218 non-null  float64
 12  NOME_MUNICIPIO_CIRCUNSCRIÇÃO  327218 non-nul

In [23]:
df["DIAS_REGISTRO"] = df["DIAS_REGISTRO"].astype(np.int64)
df["HORA_OCORRENCIA_BO"] = df["HORA_OCORRENCIA_BO"].astype(np.int64)
df["NUMERO_LOGRADOURO"] = df["NUMERO_LOGRADOURO"].astype(np.int64)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327218 entries, 5 to 618396
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   NOME_MUNICIPIO                327218 non-null  object 
 1   DIAS_REGISTRO                 327218 non-null  int64  
 2   MES_REGISTRO                  327218 non-null  int64  
 3   DIA_REGISTRO                  327218 non-null  int64  
 4   MES_OCORRENCIA                327218 non-null  int64  
 5   DIA_OCORRENCIA                327218 non-null  int64  
 6   HORA_OCORRENCIA_BO            327218 non-null  int64  
 7   BAIRRO                        327218 non-null  object 
 8   LOGRADOURO                    327218 non-null  object 
 9   NUMERO_LOGRADOURO             327218 non-null  int64  
 10  LATITUDE                      327218 non-null  float64
 11  LONGITUDE                     327218 non-null  float64
 12  NOME_MUNICIPIO_CIRCUNSCRIÇÃO  327218 non-nul

## 3.8 Modularizando coordenadas geográficas

In [25]:
df["LATITUDE"] = df["LATITUDE"].abs()
df["LONGITUDE"] = df["LONGITUDE"].abs()

In [26]:
df.head(3)

,NOME_MUNICIPIO,DIAS_REGISTRO,MES_REGISTRO,DIA_REGISTRO,MES_OCORRENCIA,DIA_OCORRENCIA,HORA_OCORRENCIA_BO,BAIRRO,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,NOME_MUNICIPIO_CIRCUNSCRIÇÃO,NATUREZA_APURADA,RUBRICA
5,S.PAULO,1,1,1,12,31,9,SE,AVENIDA DO ESTADO,4021,23.552728,46.626623,S.PAULO,FURTO - OUTROS,Furto (art. 155)
8,S.PAULO,0,1,1,1,1,7,BRÁS,AVENIDA DO ESTADO,4127,23.551932,46.625661,S.PAULO,FURTO - OUTROS,Furto (art. 155)
10,S.PAULO,0,12,31,12,31,4,SE,RUA DO GLICERIO,90,23.553375,46.627812,S.PAULO,FURTO - OUTROS,Furto (art. 155)


## 3.9 Excluindo atributos sobressalentes após experimentos iniciais

In [27]:
df.drop(["NOME_MUNICIPIO", "BAIRRO", "LOGRADOURO","NOME_MUNICIPIO_CIRCUNSCRIÇÃO", "NATUREZA_APURADA" ], axis=1, inplace=True)

In [28]:
df.head(3)

,DIAS_REGISTRO,MES_REGISTRO,DIA_REGISTRO,MES_OCORRENCIA,DIA_OCORRENCIA,HORA_OCORRENCIA_BO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,RUBRICA
5,1,1,1,12,31,9,4021,23.552728,46.626623,Furto (art. 155)
8,0,1,1,1,1,7,4127,23.551932,46.625661,Furto (art. 155)
10,0,12,31,12,31,4,90,23.553375,46.627812,Furto (art. 155)


## 3.10 Aplicando a função LabelEnconder ao atributo alvo

In [29]:
labelenconder = LabelEncoder()

df["CLASSIFICACAO"] = labelenconder.fit_transform(df["RUBRICA"])


In [30]:
df.head(3)

,DIAS_REGISTRO,MES_REGISTRO,DIA_REGISTRO,MES_OCORRENCIA,DIA_OCORRENCIA,HORA_OCORRENCIA_BO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,RUBRICA,CLASSIFICACAO
5,1,1,1,12,31,9,4021,23.552728,46.626623,Furto (art. 155),21
8,0,1,1,1,1,7,4127,23.551932,46.625661,Furto (art. 155),21
10,0,12,31,12,31,4,90,23.553375,46.627812,Furto (art. 155),21


## 3.11 Organizando os dados

In [31]:
df = df[['DIAS_REGISTRO', 'MES_REGISTRO', 'DIA_REGISTRO', 'MES_OCORRENCIA', 'DIA_OCORRENCIA',
        "HORA_OCORRENCIA_BO", 'LATITUDE', 'LONGITUDE',
        'CLASSIFICACAO','RUBRICA']]

In [32]:
df.head(3)

,DIAS_REGISTRO,MES_REGISTRO,DIA_REGISTRO,MES_OCORRENCIA,DIA_OCORRENCIA,HORA_OCORRENCIA_BO,LATITUDE,LONGITUDE,CLASSIFICACAO,RUBRICA
5,1,1,1,12,31,9,23.552728,46.626623,21,Furto (art. 155)
8,0,1,1,1,1,7,23.551932,46.625661,21,Furto (art. 155)
10,0,12,31,12,31,4,23.553375,46.627812,21,Furto (art. 155)


## 3.12 Diminuindo a quantidade de atributos alvo

In [33]:
contador = df["RUBRICA"].value_counts() 
df.value_counts()

DIAS_REGISTRO  MES_REGISTRO  DIA_REGISTRO  MES_OCORRENCIA  DIA_OCORRENCIA  HORA_OCORRENCIA_BO  LATITUDE   LONGITUDE  CLASSIFICACAO  RUBRICA                                                                   
0              6             10            6               10              12                  22.729741  47.252680  21             Furto (art. 155)                                                              14
               3             31            3               31              12                  23.945150  46.313258  15             Drogas para consumo pessoal sem autorização ou em desacordo (Art.28,caput)    10
                                                                           11                  23.945150  46.313258  15             Drogas para consumo pessoal sem autorização ou em desacordo (Art.28,caput)     9
               6             10            6               10              3                   22.729741  47.252680  21             Furto (art. 155)      

In [34]:
df  = df[df["RUBRICA"].isin(df["RUBRICA"].value_counts()[df["RUBRICA"].value_counts() > 200].index)]

In [35]:
df["RUBRICA"].value_counts()

Furto (art. 155)                                                                    137108
Roubo (art. 157)                                                                    106804
Lesão corporal culposa na direção de veículo automotor (Art. 303)                    29101
Lesão corporal (art. 129)                                                            28187
Drogas sem autorização ou em desacordo (Art.33, caput)                               13639
Drogas para consumo pessoal sem autorização ou em desacordo (Art.28,caput)            6000
Homicídio (art. 121)                                                                  1962
Associarem-se duas ou mais pessoas - arts. 33, caput e § 1o, e 34 (Art.35,caput)      1115
Homicídio culposo na direção de veículo automotor (Art. 302)                          1035
Porte ilegal de arma de fogo de uso permitido (Art. 14)                                854
Posse ou porte ilegal de arma de fogo de uso restrito (Art. 16)                        645

## 3.13 Balanceado os dados 

In [36]:
teste = df.sample(100000)

teste

,DIAS_REGISTRO,MES_REGISTRO,DIA_REGISTRO,MES_OCORRENCIA,DIA_OCORRENCIA,HORA_OCORRENCIA_BO,LATITUDE,LONGITUDE,CLASSIFICACAO,RUBRICA
425727,0,5,1,5,1,12,23.583695,46.667371,21,Furto (art. 155)
287011,3,3,7,3,4,4,22.787094,47.299026,28,Lesão corporal culposa na direção de veículo a...
297317,0,3,29,3,29,3,23.391669,46.282985,36,Roubo (art. 157)
443656,0,5,31,5,31,2,23.712943,46.698560,21,Furto (art. 155)
85559,1,1,8,1,7,7,23.592098,46.832799,21,Furto (art. 155)
...,...,...,...,...,...,...,...,...,...,...
19120,0,1,24,1,24,7,23.634334,46.725488,36,Roubo (art. 157)
132671,4,2,6,2,2,11,23.626547,46.710285,21,Furto (art. 155)
374110,5,4,11,4,6,6,20.211492,50.927189,28,Lesão corporal culposa na direção de veículo a...
50257,0,1,8,1,8,4,22.593498,46.524964,27,Lesão corporal (art. 129)


In [37]:
teste["RUBRICA"].value_counts()

Furto (art. 155)                                                                    41823
Roubo (art. 157)                                                                    32604
Lesão corporal culposa na direção de veículo automotor (Art. 303)                    9004
Lesão corporal (art. 129)                                                            8648
Drogas sem autorização ou em desacordo (Art.33, caput)                               4171
Drogas para consumo pessoal sem autorização ou em desacordo (Art.28,caput)           1902
Homicídio (art. 121)                                                                  601
Associarem-se duas ou mais pessoas - arts. 33, caput e § 1o, e 34 (Art.35,caput)      344
Homicídio culposo na direção de veículo automotor (Art. 302)                          299
Porte ilegal de arma de fogo de uso permitido (Art. 14)                               243
Posse ou porte ilegal de arma de fogo de uso restrito (Art. 16)                       190
Porte de a

# 4. Treinando o modelo

## 4.1 Separando atributos base e atributos alvo

In [38]:
X = df.iloc[:, : -2]
y = df.loc[:, "CLASSIFICACAO"]
X

,DIAS_REGISTRO,MES_REGISTRO,DIA_REGISTRO,MES_OCORRENCIA,DIA_OCORRENCIA,HORA_OCORRENCIA_BO,LATITUDE,LONGITUDE
5,1,1,1,12,31,9,23.552728,46.626623
8,0,1,1,1,1,7,23.551932,46.625661
10,0,12,31,12,31,4,23.553375,46.627812
11,1,1,1,12,31,8,23.552688,46.626680
12,1,1,1,12,31,8,23.551201,46.634047
...,...,...,...,...,...,...,...,...
618392,0,6,7,6,7,2,23.629124,46.788386
618393,0,6,13,6,13,1,23.641856,46.804309
618394,0,6,15,6,15,1,23.626038,46.794801
618395,0,6,21,6,21,11,23.641856,46.804309


## 4.2 Separando conjunto de teste e treinamento

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=1)

In [40]:
len(X_train), len (X_test), len(y_train), len(y_test)

(261584, 65397, 261584, 65397)

## 4.3 Instanciando o treinamento do modelo Categorical Naive Bayes

In [41]:
cnb = CategoricalNB()
cnb.fit(X_train, y_train)

CategoricalNB()

## 4.4 Checando métricas

In [42]:
round(cnb.score(X_train, y_train),4)

0.4512

In [43]:
cnb.score(X_test, y_test)

0.44697769010810895

## 4.5 Instanciando o treinamento do modelo random forest

In [44]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [45]:
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

## 4.6 Checando métricas

In [48]:
rf.score(X_train, y_train)

0.9933482170163312

In [47]:
rf.score(X_test, y_test)

0.49909017233206415